In [236]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET
import numpy as np

In [237]:
buy_out = df(columns=["date",
                      "phone_number",
                      "notebook",
                      "tablet",
                      "phone",
                      "spare_part",
                      "conversation",
                      "customer_price",
                      "our_price",
                      "recycle",
                      "customer_decision"])


buy_out

,date,phone_number,notebook,tablet,phone,spare_part,conversation,customer_price,our_price,recycle,customer_decision


In [238]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s");
        text = "";
        for i in source_sentence.itertext():
            text += i;
        dict_of_leads[id] = text;
    return dict_of_leads;

make_dict_of_leads()["46"]

'89140580517 продажа айфон6s на выезде Ляпидевского 12.'

In [243]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

def make_common_table():    
    calls = df(columns=[
            "CustomerPhone_Phone",
            "CustomerPlace_Place",
            "Buy_Word",
            "Pawn_Word",
            "Notebook_Word",
            "Notebook_Vendor",
            "Notebook_Model",
            "Notebook_UserDefinedName",
            "Phone_Word",
            "Phone_Vendor",
            "Phone_Model",
            "Phone_UserDefinedName",
            "source_sentence"
        ])

    facts = compare_facts_to_leads()
    leads = make_dict_of_leads()
    for lead in facts:
        elems = facts[lead]
        one_sentence = lead
        cols = []
        values = []
        for fact_name in elems:
            for fact_field in fact_name:
                cols.append(fact_name.tag + "_" + fact_field.tag)
                values.append(fact_field.attrib["val"])
        one_row = pd.DataFrame([values], columns=cols)
        calls = calls.append(one_row)

        values = []
        cols = []
    return calls;

calls = make_common_table()

pawn = calls[calls.Pawn_Word.notnull()]
buy_out = calls[calls.Buy_Word.notnull()]
buy_out.to_excel("buy_out.xlsx")
pawn.to_excel("pawn.xlsx")

to_str = lambda x: " " + x if x is not np.nan else ""
calls["notebook"] = calls["Notebook_Word"] + calls["Notebook_Vendor"].map(to_str) + calls["Notebook_Model"].map(to_str)
calls["phone"] = calls["Phone_Word"] + calls["Phone_Vendor"].map(to_str) + calls["Phone_Model"].map(to_str)

calls.drop(['Notebook_Word', 'Notebook_Vendor', 'Notebook_Model'], axis=1, inplace=True)
calls.drop(['Phone_Word', 'Phone_Vendor', 'Phone_Model'], axis=1, inplace=True)

calls.to_excel("whole_table.xlsx")